In [1]:
import numpy as np
import optuna
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import xgboost as xgb

In [2]:
with open('processed_data/preprocessed_LSTM_CNN.pkl', 'rb') as f:
    LSTM_CNN_preprocessed_data = pickle.load(f)


with open('processed_data/preprocessed_XGboost.pkl', 'rb') as f:
    XGBoost_preprocessed_data = pickle.load(f)

In [3]:
# The variable LSTM_CNN_preprocessed_data is a list of our 5 bins [0-4][][]. Each bin contains the 15 belonging stocks or ETFs [0-4][0-14][].
# After accessing the stock we can get the we are left with a 2 element list containing [0] X the features and [1] y the target [0-4][0-14][0-1].

# For the NNs the accessed data is 3D (samples, sample_size, features_per_sample)

LSTM_CNN_preprocessed_data[2][7][0].shape
# This for example accesses the X data from the 8th stock in bin 3.

(1228, 61)

In [4]:
# The variable XGBoost_preprocessed_data is a list of our 5 bins [0-4][][]. Each bin contains the 15 belonging stocks or ETFs [0-4][0-14][].
# After accessing the stock we can get X_train, y_train, X_val, y_val, X_test, y_test [0-4][0-14][0-5].

# For the Tree based model the accessed data is 2D (sample_size, features)

XGBoost_preprocessed_data[0][0][0].shape
# This accesses the X_train set of the first stock in bin 1.

(972, 247)

In [5]:
# Only pass pandas DataFrames as data_X data_y to the function below.
def create_windows(data_X, data_y, window):
    X, y, indices = [], [], []
    for i in range(window, len(data_X)):
        X.append(data_X[i - window:i].values.astype("float32"))
        y.append(float(data_y.iloc[i-1])) # i-1 because we shifted the target by one in the General preprocessing pipeline 
        indices.append(data_y.index[i-1])
    return np.array(X, dtype="float32"), np.array(y, dtype="float32"), np.array(indices)

# -------------------------------------------------------------------
# 2. OBJECTIVE: TUNE AN LSTM (REGRESSION)
# -------------------------------------------------------------------
def make_lstm_objective(X, y):
    def lstm_objective(trial):
        # Suggest hyperparameters
        n_units = trial.suggest_int("lstm_units", 32, 128, step=32)
        n_layers = trial.suggest_int("lstm_layers", 1, 2, step=1)
        dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5, step=0.1)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
        batch_size = trial.suggest_categorical("batch_size", [16, 32])
        n_dense = trial.suggest_int("dense_units", 16, 128, step=16)
        window_size = trial.suggest_int("window_size", 10, 60, step=10)

        # Start create windows
        X_windows, y_windows, indices = create_windows(X, y, window_size)

        end_train_set = X.index[int(X.shape[0] * 0.8)]
        end_validation_set = X.index[int(X.shape[0] * 0.9)]

        train_mask = indices < end_train_set
        validation_mask = (indices >= end_train_set) & (indices < end_validation_set)
        test_mask = indices >= end_validation_set

        X_train = X_windows[train_mask]
        y_train = y_windows[train_mask]
        X_val = X_windows[validation_mask]
        y_val = y_windows[validation_mask]
        X_test = X_windows[test_mask]
        y_test = y_windows[test_mask]
        # End create windows

        # Build model
        model = Sequential()
        for i in range(n_layers):
            bool_return_sequences = (i < n_layers - 1)  # Return sequences for all but the last layer
            bool_firtst_layer = (i == 0)  # First layer needs input shape
            if bool_firtst_layer:
                model.add(LSTM(units=n_units, input_shape=(window_size, X_train.shape[2]),
                            return_sequences=bool_return_sequences))
            else:
                model.add(LSTM(units=n_units, return_sequences=bool_return_sequences))
            model.add(Dropout(rate=dropout_rate))
        model.add(Dense(1, activation="linear"))  # regression

        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss="mse")

        # Early stopping callback to prune unpromising trials
        early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

        # Train
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=batch_size,
            callbacks=[early_stop],
            verbose=0
        )

        # Evaluate on validation set
        y_pred = model.predict(X_test, batch_size=batch_size)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_pred - y_test) / y_test)) * 100
        nrmse = np.sqrt(mean_squared_error(y_test, y_pred)) / np.mean(y_test) * 100  # in percent

        trial.set_user_attr("mae", mae)
        trial.set_user_attr("r2", r2)
        trial.set_user_attr("rmse", rmse)
        trial.set_user_attr("nrmse", nrmse)

        return mape
    return lstm_objective

In [7]:
# Run the study

LSTM_study_results = []

N_TRIALS = 70
for i in range(4, len(LSTM_CNN_preprocessed_data)):

    LSTM_bin_results = []

    for j in range(4, len(LSTM_CNN_preprocessed_data[i])):

        print(f"Running LSTM study for bin {i+1}, stock {j+1}...")
        X, y = LSTM_CNN_preprocessed_data[i][j]
        X = X.drop(columns=["usd_jpy"])  # Drop the ticker smybol

        # 5.1. LSTM study
        lstm_objective = make_lstm_objective(X, y)
        lstm_study = optuna.create_study(
            direction="minimize",
            study_name="lstm_regression_study",
            sampler=optuna.samplers.TPESampler(),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
        )
        lstm_study.optimize(lstm_objective, n_trials=N_TRIALS)

        results = {
            "best_trial_value": lstm_study.best_trial.value,
            "best_trial_params": lstm_study.best_trial.params,
            "mae": lstm_study.best_trial.user_attrs.get("mae", None),
            "r2": lstm_study.best_trial.user_attrs.get("r2", None),
            "rmse": lstm_study.best_trial.user_attrs.get("rmse", None),
            "nrmse": lstm_study.best_trial.user_attrs.get("nrmse", None)
        }

        LSTM_bin_results.append(results)
        # Save the stock
        with open(f'results/LSTM/bin{i+1}/Performance_metrices_LSTM_bin_{i+1}_stock_{j+1}.pkl', 'wb') as f:
            pickle.dump(results, f)
    # Save the bin
    with open(f'results/LSTM/Performance_metrices_LSTM_bin_{i+1}.pkl', 'wb') as f:
        pickle.dump(LSTM_bin_results, f)

[I 2025-06-12 12:18:49,852] A new study created in memory with name: lstm_regression_study


Running LSTM study for bin 5, stock 5...


c:\Users\jerem\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


[I 2025-06-12 12:19:23,383] Trial 0 finished with value: 22.687973022460938 and parameters: {'lstm_units': 64, 'lstm_layers': 1, 'dropout_rate': 0.4, 'learning_rate': 0.00018637876949370713, 'batch_size': 32, 'dense_units': 80, 'window_size': 60}. Best is trial 0 with value: 22.687973022460938.
c:\Users\jerem\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

[I 2025-06-12 22:49:49,131] Trial 67 finished with value: 15.110980033874512 and parameters: {'lstm_units': 64, 'lstm_layers': 2, 'dropout_rate': 0.4, 'learning_rate': 0.0005594441809139488, 'batch_size': 32, 'dense_units': 64, 'window_size': 10}. Best is trial 42 with value: 14.437326431274414.
c:\Users\jerem\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step


[I 2025-06-12 22:51:04,003] Trial 68 finished with value: 17.032011032104492 and parameters: {'lstm_units': 64, 'lstm_layers': 2, 'dropout_rate': 0.30000000000000004, 'learning_rate': 0.00039030659353672724, 'batch_size': 16, 'dense_units': 112, 'window_size': 10}. Best is trial 42 with value: 14.437326431274414.
c:\Users\jerem\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step


[I 2025-06-12 22:51:56,692] Trial 69 finished with value: 29.058765411376953 and parameters: {'lstm_units': 32, 'lstm_layers': 2, 'dropout_rate': 0.30000000000000004, 'learning_rate': 0.0005412182848252382, 'batch_size': 16, 'dense_units': 128, 'window_size': 10}. Best is trial 42 with value: 14.437326431274414.


In [ ]:
# -------------------------------------------------------------------
# 4. OBJECTIVE: TUNE XGBOOST (REGRESSION)
# -------------------------------------------------------------------
def make_xgb_objective(entry):
    X_train, y_train, X_val, y_val, X_test, y_test = entry
    def xgb_objective(trial):
        # Suggest hyperparameters
        param = {
            "verbosity": 0,
            "objective": "reg:squarederror",
            "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
            "lambda": trial.suggest_loguniform("lambda", 1e-3, 10.0),
            "alpha": trial.suggest_loguniform("alpha", 1e-3, 10.0),
        }

        if param["booster"] == "gbtree" or param["booster"] == "dart":
            param["max_depth"] = trial.suggest_int("max_depth", 3, 10)
            param["eta"] = trial.suggest_loguniform("eta", 1e-4, 1e-1)
            param["gamma"] = trial.suggest_float("gamma", 0, 5)
            param["subsample"] = trial.suggest_float("subsample", 0.5, 1.0)
            param["colsample_bytree"] = trial.suggest_float("colsample_bytree", 0.5, 1.0)

        if param["booster"] == "dart":
            param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
            param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
            param["rate_drop"] = trial.suggest_float("rate_drop", 0.0, 0.3)
            param["skip_drop"] = trial.suggest_float("skip_drop", 0.0, 0.3)

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        dtest = xgb.DMatrix(X_test, label=y_test)

        pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse")

        # Train with early stopping and pruning
        bst = xgb.train(
            param,
            dtrain,
            num_boost_round=1000,
            evals=[(dval, "validation")],
            early_stopping_rounds=20,
            callbacks=[pruning_callback],
            verbose_eval=False
        )

        y_pred = bst.predict(dtest)
        val_rmse = np.sqrt(mean_squared_error(y_val, y_pred))
                # Evaluate on validation set
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)

        trial.set_user_attr("mae", mae)
        trial.set_user_attr("r2", r2)



        return val_rmse
    return xgb_objective

In [ ]:
# 5.3. XGBoost study
N_TRIALS = 100

XGBoost_data_results = []
for i in range(len(XGBoost_preprocessed_data)):
    XGBoost_bin_results = []
    for j in range(len(XGBoost_preprocessed_data[i])):
        X_train, y_train, X_val, y_val, X_test, y_test = XGBoost_preprocessed_data[i][j]
        xgb_study = optuna.create_study(
            direction="minimize",
            study_name="xgb_regression_study",
            sampler=optuna.samplers.TPESampler(),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
        )
        xgb_study.optimize(make_xgb_objective(X_train, y_train, X_val, y_val, X_test, y_test), n_trials=N_TRIALS)

        results = {
            "best_val": xgb_study.best_trial.value,
            "best_params": xgb_study.best_trial.params,
            "mae": lstm_study.best_trial.user_attrs.get("mae", None),
            "r2": lstm_study.best_trial.user_attrs.get("r2", None)
        }
        XGBoost_bin_results.append(results)
    # Save the data
    with open(f'results/XGBoost/Performance_metrices_XGBoost_bin_{i}.pkl', 'wb') as f:
        pickle.dump(XGBoost_bin_results, f)
    XGBoost_data_results.append(XGBoost_bin_results)
# Save the data
with open(f'results/XGBoost/Performance_metrices_XGBoost_full.pkl', 'wb') as f:
    pickle.dump(XGBoost_data_results, f)

In [ ]:
# Only pass pandas DataFrames as data_X data_y to the function below.
def create_windows(data_X, data_y, window_size):
    X, y, indices = [], [], []
    for i in range(window_size, len(data_X)):
        X.append(data_X[i - window_size:i].values.astype("float32"))
        y.append(float(data_y.iloc[i-1])) # i-1 because we shifted the target by one in the General preprocessing pipeline 
        indices.append(data_y.index[i-1])
    return np.array(X, dtype="float32"), np.array(y, dtype="float32"), np.array(indices)

# -------------------------------------------------------------------
# 3. OBJECTIVE: TUNE A 1D‐CNN (REGRESSION)
# -------------------------------------------------------------------
def make_cnn_objective(X, y):
    def cnn_objective(trial):
        # Suggest hyperparameters
        n_filters = trial.suggest_int("n_filters", 16, 128, step=16)
        kernel_size = trial.suggest_int("kernel_size", 2, 8)
        pool_size = trial.suggest_int("pool_size", 2, 4)
        dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5, step=0.1)
        learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
        batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
        dense_units = trial.suggest_int("dense_units", 16, 128, step=16)
        window_size = trial.suggest_int("window_size", 10, 60, step=10)


        # Start create windows
        X_windows, y_windows, indices = create_windows(X, y, window_size)

        end_train_set = X.index[int(X.shape[0] * 0.8)]
        end_validation_set = X.index[int(X.shape[0] * 0.9)]

        train_mask = indices < end_train_set
        validation_mask = (indices >= end_train_set) & (indices < end_validation_set)
        test_mask = indices >= end_validation_set

        X_train = X_windows[train_mask]
        y_train = y_windows[train_mask]
        X_val = X_windows[validation_mask]
        y_val = y_windows[validation_mask]
        X_test = X_windows[test_mask]
        y_test = y_windows[test_mask]
        # End create windows



        # Build model
        model = Sequential()
        model.add(Conv1D(filters=n_filters, kernel_size=kernel_size, activation="relu",
                        input_shape=(X_train[1], X_train[2])))
        model.add(MaxPooling1D(pool_size=pool_size))
        model.add(Dropout(rate=dropout_rate))
        model.add(Flatten())
        model.add(Dense(units=dense_units, activation="relu"))
        model.add(Dense(1, activation="linear"))

        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss="mse")

        early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

        # Train
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=batch_size,
            callbacks=[early_stop],
            verbose=0
        )

        y_pred = model.predict(X_test, batch_size=batch_size)
        val_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        trial.set_user_attr("mae", mae)
        trial.set_user_attr("r2", r2)
        
        return val_rmse
    return cnn_objective


In [ ]:
N_trials = 100

CNN_data_results = []

for i in range(len(LSTM_CNN_preprocessed_data)):
    
    CNN_bin_results = []
    for j in range(len(LSTM_CNN_preprocessed_data[i])):
        X, y = LSTM_CNN_preprocessed_data[i][j]


        # 5.2. CNN study
        study = optuna.create_study(
            direction="minimize",
            study_name="cnn_regression_study",
            sampler=optuna.samplers.TPESampler(),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
        )
        study.optimize(make_cnn_objective(X, y), n_trials=N_TRIALS)

        results = {
            "best_val": study.best_trial.value,
            "best_params": study.best_trial.params,
            "mae": study.best_trial.user_attrs.get("mae", None),
            "r2": study.best_trial.user_attrs.get("r2", None)
        }


        CNN_bin_results.append(results)
    # Save the data
    with open(f'results/CNN/Performance_metrices_CNN_bin_{i}.pkl', 'wb') as f:
        pickle.dump(CNN_bin_results, f)
    CNN_data_results.append(CNN_bin_results)
# Save the data
with open(f'results/CNN/Performance_metrices_CNN_full.pkl', 'wb') as f:
    pickle.dump(CNN_data_results, f)

KeyboardInterrupt: 